In [1]:
import numpy as np
from scipy import sparse

In [2]:
tol_var = 10**-10

In [3]:
# Function for power iteration taking matrices as input
def power_iteration(s_matrix):
    one_vec = np.ones((tot_num_of_nodes,1))
    counter = 0
    # initialise the r_0
    r_var = 1/ float(tot_num_of_nodes) * one_vec
    while True:
        temp_1 = (1-0.8)/tot_num_of_nodes
        temp_2 = 0.8 * s_matrix.dot(r_var)
        r_new_var = temp_1 * one_vec + temp_2
        L_1_Norm = np.linalg.norm(abs(r_new_var - r_var), ord=1)
        if L_1_Norm < tol_var:
            break
        
        r_var = r_new_var
        counter = counter + 1
    
    print("For tolerance - ",tol_var,"Total number of iterations for power iteration it took is :- ", counter)
    print("\n")
    return r_var

In [10]:
def getTopK(k_top,r):
    # Top k page ranks with values
    k_top_Pages_id = r.T[0].argsort()[-k_top:][::-1]
    k_top_Page_list = []
    for ind in k_top_Pages_id:
        k_top_Page_list.append((r[ind][0], ind+1))
    
    # Printing top k nodes with scores
    print("Top {} nodes".format(k_top))
    for pr in enumerate(k_top_Page_list):
        (ind, (pRVal, nodeId)) = pr
        print("Page rank ", ind+1, "for page id ",nodeId+1,"with value is - ", pRVal)

    print("\n\n")

In [11]:
def getBottomK(k_bottom,r):
    # Bottom 5 page ranks with values
    k_bottom_Pages_id = r.T[0].argsort()[:k_bottom]
    k_bottom_Page_list = []
    for ind in k_bottom_Pages_id:
        k_bottom_Page_list.append((r[ind][0], ind+1))
    print("Bottom {} nodes".format(k_bottom))
    for pr in enumerate(k_bottom_Page_list):
        (ind, (pRVal, nodeId)) = pr
        print("Weighted page rank value", ind+1, "for page id ",nodeId+1,"is - ", pRVal)
    
    print("\n\n")

In [6]:
# Code to get graph data as graph edges.
edges_of_graph = dict()

with open("page_rank/parent_child_file.txt") as file:
    line = file.readline()
    while line:
        line_list = []
        for node in line.strip().split(" "):
            line_list.append(int(node))
        if len(set(line_list)) != 1:
            if edges_of_graph.get(line_list[0]):
                edges_of_graph[line_list[0]].add(line_list[1])
            else:
                edges_of_graph[line_list[0]] = {line_list[1]}
        line = file.readline()

#print(edges_of_graph)
tot_num_of_nodes = max(sorted(edges_of_graph.keys(), reverse=True)[:1][0],
                       sorted(edges_of_graph.keys(), reverse=True)[:1][0])
print("Total number of nodes being used are ", tot_num_of_nodes)

Total number of nodes being used are  108658


In [7]:
# code to get weighted sparse matrix from graph
row_list = []
col_list = []
data_list = []

for node in edges_of_graph.keys():
    node_vals = edges_of_graph.get(node)
    node_cnt = len(node_vals)
    each_n_val = 1 / node_cnt
    
    values_dict = dict()
    for value in node_vals:
        if values_dict.get(value):
            values_dict[value] = values_dict.get(value) + 1
        else:
            values_dict[value] = 1
    for n_val in list(set(node_vals)):
        cnt = values_dict.get(n_val)
        data_list.append(cnt)
        row_list.append(n_val-1)
        col_list.append(node-1)
        
matrix = sparse.csr_matrix((data_list, (row_list, col_list)), shape=(tot_num_of_nodes, tot_num_of_nodes))

r_list = []
c_list = []
data_list = []
r_sum = matrix.sum(axis=1).T.tolist()[0]
c_sum = matrix.sum(axis=0).tolist()[0]

for node in edges_of_graph.keys():
    node_vals = edges_of_graph.get(node)
    
    n_sum_in = 0
    for n_val in list(set(node_vals)):
        n_sum_in = n_sum_in + r_sum[n_val-1]
        
    n_sum_out = 0
    for n_val in list(set(node_vals)):
        n_sum_out = n_sum_in + c_sum[n_val-1]
        
    for n_val in list(set(node_vals)):
        n_in_c = r_sum[n_val-1]
        w_in = float(n_in_c/n_sum_in)
        n_out_c = c_sum[n_val-1]
        w_out =float(n_out_c/n_sum_out)
        r_list.append(n_val-1)
        c_list.append(node-1)
        weight = w_in * w_out
        data_list.append(weight)
    
Weighted_Sparse_Matrix = sparse.csr_matrix((data_list, (r_list, c_list)), shape=(tot_num_of_nodes, tot_num_of_nodes))
print("-----------------------Weighted Sparse Matrix------------------------")
print(Weighted_Sparse_Matrix)


-----------------------Weighted Sparse Matrix------------------------
  (1, 0)	0.007524858908895459
  (2, 0)	0.007524858908895459
  (3, 0)	0.007524858908895459
  (4, 0)	0.007524858908895459
  (5, 0)	0.007524858908895459
  (6, 0)	0.007524858908895459
  (7, 0)	0.011287288363343188
  (8, 0)	0.007524858908895459
  (9, 0)	0.007524858908895459
  (10, 0)	0.007524858908895459
  (11, 0)	0.007524858908895459
  (12, 0)	0.00806234883095942
  (13, 0)	0.007524858908895459
  (14, 0)	0.007524858908895459
  (15, 0)	0.007524858908895459
  (16, 0)	0.007524858908895459
  (17, 0)	0.007524858908895459
  (18, 0)	0.007524858908895459
  (19, 0)	0.007524858908895459
  (20, 0)	0.007524858908895459
  (21, 0)	0.007524858908895459
  (22, 0)	0.007524858908895459
  (23, 0)	0.007524858908895459
  (24, 0)	0.007524858908895459
  (25, 0)	0.007524858908895459
  :	:
  (108632, 108608)	0.0
  (108633, 108657)	0.9583333333333334
  (108634, 108633)	0.0
  (108635, 108633)	0.0
  (108636, 108633)	0.0
  (108637, 108633)	0.0
  (108

In [12]:
# Using weighted page rank and power iteration
r = power_iteration(Weighted_Sparse_Matrix)
print("weighted page rank with power iteration :- ")
getTopK(10, r)
getBottomK(10, r)


For tolerance -  1e-10 Total number of iterations for power iteration it took is :-  3


weighted page rank with power iteration :- 
Top 10 nodes
Page rank  1 for page id  108635 with value is -  3.2517930877922773e-06
Page rank  2 for page id  103571 with value is -  2.5202576326285576e-06
Page rank  3 for page id  1329 with value is -  2.322396314818146e-06
Page rank  4 for page id  75430 with value is -  2.15113725270977e-06
Page rank  5 for page id  62696 with value is -  2.0842002278284846e-06
Page rank  6 for page id  1357 with value is -  1.890483694677975e-06
Page rank  7 for page id  15617 with value is -  1.880132123692529e-06
Page rank  8 for page id  1358 with value is -  1.8795949681100727e-06
Page rank  9 for page id  25892 with value is -  1.8755128355409677e-06
Page rank  10 for page id  1361 with value is -  1.8729286630167052e-06



Bottom 10 nodes
Weighted page rank value 1 for page id  2 is -  1.840637596863553e-06
Weighted page rank value 2 for page id  72445 is - 